In [2]:
# from Multi_head_crossattn import MultiHeadCrossAttention
from Multi_head_selfattn import MultiHeadSelfAttention
from feedforward import FeedForward
import torch.nn as nn
import torch.nn.functional as F
import torch

In [11]:
n_layers = 5
n_heads = 8
batch_size = 32
sequence_len = 256
n_embedding = 512

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class TransformerEncoderLayer(nn.Module):
  def __init__(self,n_heads,batch_size,sequence_len, n_embedding):
    super().__init__()
    self.n_heads = n_heads
    self.batch_size = batch_size
    self.sequence_len = sequence_len
    self.n_embedding = n_embedding
    self.qkv_layer = nn.Linear(n_embedding,3*self.n_embedding)
    self.dropout1 = nn.Dropout(0.1)
    self.attention = MultiHeadSelfAttention()
    self.layer_norm1 = nn.LayerNorm(n_embedding)
    self.fcl = nn.Linear(n_embedding,n_embedding)
    self.dropout2 = nn.Dropout(0.1)
    self.layer_norm2 = nn.LayerNorm(n_embedding)

  def forward(self,x):
    residual = x
    x = self.qkv_layer(x)
    x=x.view(self.batch_size,self.sequence_len,self.n_heads,3*self.n_embedding//self.n_heads)
    x = x.permute(0,2,1,3)
    x = self.attention(x)
    print(x.shape)
    x = x.reshape(self.batch_size,self.sequence_len,self.n_embedding)
    x = self.dropout1(x)
    print(residual.shape)
    x = x + residual
    x = self.layer_norm1(x)
    x = self.fcl(x)
    x = self.dropout2(x)
    x = x + residual
    x = self.layer_norm2(x)
    return x

class TransformerEncoder(nn.Module):
  def __init__(self,n_layers,n_heads,batch_size,sequence_len, n_embedding):
    super().__init__()
    self.layers = nn.ModuleList([TransformerEncoderLayer(n_heads,batch_size,sequence_len, n_embedding) for _ in range(n_layers)])

  def forward(self,x):
    for idx , layer in enumerate(self.layers):
      print(f'Layer : {idx + 1}')
      x = layer(x)
    return x




Using device: cpu


In [12]:

encoder = TransformerEncoder(n_layers,n_heads,batch_size,sequence_len,n_embedding)
output = encoder(torch.randn(batch_size,sequence_len,n_embedding))
# print(output.shape)

Layer : 1
torch.Size([32, 8, 256, 64])
torch.Size([32, 256, 512])
Layer : 2
torch.Size([32, 8, 256, 64])
torch.Size([32, 256, 512])
Layer : 3
torch.Size([32, 8, 256, 64])
torch.Size([32, 256, 512])
Layer : 4
torch.Size([32, 8, 256, 64])
torch.Size([32, 256, 512])
Layer : 5
torch.Size([32, 8, 256, 64])
torch.Size([32, 256, 512])
